In [14]:
import torch
from sklearn.metrics import f1_score
from graph_qa_dataset import GraphQADataset
from torch_geometric.loader import DataListLoader
from model.graph_qa_model import GAT, HGT

In [15]:
train_dataset = GraphQADataset(split="train", data_size=20)
val_dataset = GraphQADataset(split="validation", data_size=2)

Found cached dataset squad (/home/mingzhe/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-10-31 03:45:15 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_met

2022-10-31 03:45:15 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-10-31 03:45:15 INFO: Use device: gpu
2022-10-31 03:45:15 INFO: Loading: tokenize
2022-10-31 03:45:15 INFO: Loading: pos
2022-10-31 03:45:16 INFO: Loading: constituency
2022-10-31 03:45:16 INFO: Done loading processors!
  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = HGT(hidden_channels=64, out_channels=2, num_heads=4, num_layers=2, metadata=train_dataset.metadata)

In [ ]:
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [ ]:
model.train()

data = train_dataset.graph_data[0][0]

print(data)

HeteroData(
  question={
    x=[27, 768],
    y=[27, 2]
  },
  context={
    x=[260, 768],
    y=[260, 2]
  },
  (question, connect, question)={ edge_index=[2, 52] },
  (context, connect, question)={ edge_index=[2, 7] },
  (context, connect, context)={ edge_index=[2, 506] },
  (question, rev_connect, context)={ edge_index=[2, 7] }
)


In [ ]:
out = model(data.x_dict, data.edge_index_dict)

In [ ]:
out

tensor([[0.0244, 0.0301],
        [0.0250, 0.0308],
        [0.0257, 0.0297],
        [0.0246, 0.0275],
        [0.0240, 0.0281],
        [0.0266, 0.0231],
        [0.0218, 0.0247],
        [0.0172, 0.0301],
        [0.0180, 0.0308],
        [0.0242, 0.0345],
        [0.0177, 0.0371],
        [0.0224, 0.0385],
        [0.0219, 0.0326],
        [0.0291, 0.0323],
        [0.0232, 0.0324],
        [0.0160, 0.0370],
        [0.0150, 0.0200],
        [0.0192, 0.0420],
        [0.0218, 0.0407],
        [0.0213, 0.0406],
        [0.0229, 0.0344],
        [0.0259, 0.0378],
        [0.0211, 0.0359],
        [0.0204, 0.0328],
        [0.0242, 0.0334],
        [0.0258, 0.0347],
        [0.0195, 0.0334],
        [0.0262, 0.0353],
        [0.0205, 0.0343],
        [0.0186, 0.0363],
        [0.0200, 0.0332],
        [0.0220, 0.0401],
        [0.0179, 0.0359],
        [0.0218, 0.0322],
        [0.0240, 0.0330],
        [0.0244, 0.0335],
        [0.0243, 0.0370],
        [0.0157, 0.0366],
        [0.0

In [ ]:
train_dataloader = DataListLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataListLoader(val_dataset, batch_size=1, shuffle=False)

In [ ]:
def train(dataloader):
    model.train()
    
    total_loss = 0
    for batch in dataloader:
        data = batch[0]
        optimizer.zero_grad()
        out = model(data.x_dict, data.edge_index_dict)

        loss = loss_op(out, data["context"].y)
        print(loss)
        total_loss += loss.item()
        loss.backward()

train(train_dataloader)

tensor(0.6900, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6900, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6900, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6900, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6900, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
